In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from time import time
import numpy as np
import os

# Evaluation Script for Virtual Segments, Compared against originally measured data


In [50]:
# Class for each small NN model
class MyModelA(nn.Module):
    def __init__(self):
        super(MyModelA, self).__init__()
        self.fc1 = nn.Linear(1, 4)
        self.fc2 = nn.Linear(4, 1)
            
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x

# Main Combined Model
# Class that uses all the small models combined and generated height , later generates all the long bones from this height
class MyEnsemble(nn.Module):
    def __init__(self, modelA, modelB, modelC, modelD, modelH2LL, modelH2UL, modelH2LA, modelH2UA):
        super(MyEnsemble, self).__init__()
        
      
        # Models that are used to predict height from detected long bone
        self.model_lower_leg = modelA
        self.model_upper_leg = modelB
        self.model_lower_arm = modelC
        self.model_upper_arm = modelD
        
        # Models for Virtual Reconstruction from height to other segments
        # Each model uses height information to predict other long bones
        
        self.height_to_lowerleg = modelH2LL
        self.height_to_upper_leg = modelH2UL
        self.height_to_lower_arm = modelH2LA
        self.height_to_upper_arm = modelH2UA
        

                
    def forward(self, x1, flag):
        
        # Function definition for height to long bone reconstruction
        def reconstruct_Segments(height):
            
            # Multipication factors are the values that were used to scale values in range (0-1)
            # Every model had different dataset, therefore also had different scaling factors
            
            pred_LL  = self.height_to_lowerleg(height/201.513) * 49.319  
            pred_UL = self.height_to_upper_leg(height/201.913) * 58.554   
            pred_LA = self.height_to_lower_arm(height/204.805) * 31.670  
            pred_UA = self.height_to_upper_arm(height/204.104) * 42.5100 
            
            #returns all the predicted long bones from height
            return pred_LL, pred_UL, pred_LA, pred_UA
        
        
        # Check the measured/detected long bone according to recieved flag and reconstruct height from respective long bone
        # Multipication factors are the scaling factors used before training each network, results in original value.
        if flag == 'lower_leg':
            x = self.model_lower_leg(x1)
            x*=195.5    
            
        if flag == 'upper_leg':
            x = self.model_upper_leg(x1)
            x*=204.1
            
        if flag == 'lower_arm':
            x = self.model_lower_arm(x1)
            x*=204.68
            
        if flag == 'upper_arm':
            x = self.model_upper_arm(x1)
            x*= 209.2665
            
        
        # After reconstructing height, pass the height to reconstruction function to reconstruct all the long bones form height.
        
        pred_LL, pred_UL, pred_LA, pred_UA = reconstruct_Segments(x)
        
        return x, pred_LL, pred_UL, pred_LA, pred_UA


In [51]:
# Create models and load state_dicts    

# Initialising 4 models for each long bone to reconstruct height
model_lower_leg = MyModelA()
model_upper_leg = MyModelA()
model_lower_arm = MyModelA()
model_upper_arm = MyModelA()

#Virtual Segments Reconstruction Model
#Initialising Models that will predict each long bone from height
model_height_to_lowerleg = MyModelA()
model_height_to_upper_leg = MyModelA()
model_height_to_lower_arm = MyModelA()
model_height_to_upper_arm = MyModelA()

# Load state dicts
# Paths to weights for each long bone models
PATH_lower_leg = 'Trained_models/lower_leg_pytorch_/ansur_model_2.6701'
PATH_upper_leg = 'Trained_models/upper_leg_model_pytorch_/upper_leg_5.84564208984375'
PATH_lower_arm = 'Trained_models/lower_arm_pytorch_/lower_arm_pytorch_3.489548921585083'
PATH_upper_arm = 'Trained_models/upper_arm_pytorch_/upper_arm_pytorch4.744106292724609'

# Virtual Models
# Paths to weights for each virtual height to long bone models
PATH_model_height_to_lowerleg =  'Virtual_Segments/Trained_models/height_lower_leg_5.075006484985352'
PATH_model_height_to_upper_leg = 'Virtual_Segments/Trained_models/height_upper_leg_18.44318962097168'
PATH_model_height_to_lower_arm = 'Virtual_Segments/Trained_models/height_lower_arm_9.933013916015625'
PATH_model_height_to_upper_arm = 'Virtual_Segments/Trained_models/Height_upper_arm_9.007071495056152'

# Loading Learned Weights for each long bone to height models
model_lower_leg.load_state_dict(torch.load(PATH_lower_leg))
model_upper_leg.load_state_dict(torch.load(PATH_upper_leg))
model_lower_arm.load_state_dict(torch.load(PATH_lower_arm))
model_upper_arm.load_state_dict(torch.load(PATH_upper_arm))

# Loading Learned Weights for height to each Virtual Segments
model_height_to_lowerleg.load_state_dict(torch.load(PATH_model_height_to_lowerleg))
model_height_to_upper_leg.load_state_dict(torch.load(PATH_model_height_to_upper_leg))
model_height_to_lower_arm.load_state_dict(torch.load(PATH_model_height_to_lower_arm))
model_height_to_upper_arm.load_state_dict(torch.load(PATH_model_height_to_upper_arm))


#Creating the main model with all the pre-trained loaded weights
model = MyEnsemble(model_lower_leg, model_upper_leg, model_lower_arm, model_upper_arm,
                    model_height_to_lowerleg, model_height_to_upper_leg,
                    model_height_to_lower_arm, model_height_to_upper_arm)

#  Function to Display Outputs with each long bone 
def display(output):
    x, pred_LL, pred_UL, pred_LA, pred_UA = output
    
    print('Stature: ', x.item(), 'LL: ', pred_LL.item(),'UL: ', pred_UL.item(),'LA: ', pred_LA.item(),'UA: ', pred_UA.item())
    


# Evaluation Lower Leg (Tibia)

In [43]:
import pandas as pd

dataset_path ='data/ansur_result.csv'
column_names = column_names = ['gender','butt_height','buttock-knee_length','buttock-popliteal_length',
                               'crotch_height','knee_height_midpatella','knee_height_sitting', 
                               'lateral-femoral_epicondyle_height', 'popliteal_height',
                               'stature', 'trochanterion_height', 'wrist_height' ]


raw_dataset = pd.read_csv(dataset_path, names=column_names)
dataset = raw_dataset.copy()

x = dataset['knee_height_midpatella']

x_train = []
y_pred = []

for index, value in enumerate(x):
    if (value != 'knee_height_midpatella') and float(value) > 375 and float(value) < 600 :
        x_train.append(float(value))
    
x_train_float = np.array(x_train[1:len(x_train)-1500])
print((len(x_train_float), len(x_test_float)))

mae = 0
mse = 0
for index, tibia in enumerate(x_train_float):
    
    lower_leg = tibia
    flag = 'lower_leg' 

    x1 = torch.from_numpy(np.array([lower_leg], dtype='float32'))
    x1.view((1,1))
    x, pred_LL, pred_UL, pred_LA, pred_UA = model(x1/592.0, flag)
    mae += abs(pred_LL - tibia/10)
    mse += (pred_LL - tibia/10)**2
    
mae/=index+1
mse/=index+1

print('Mean Absolute Error on ', index, 'samples :', mae.item() )
print('Mean Squared Error on ', index, 'samples :', np.sqrt(mse.item()) )


(2478, 1500)
Height predicted from lower_leg : is 152.54769897460938 cm
Height predicted from lower_leg : is 157.56829833984375 cm
Height predicted from lower_leg : is 177.44143676757812 cm
Height predicted from lower_leg : is 159.6602020263672 cm
Height predicted from lower_leg : is 155.68556213378906 cm
Height predicted from lower_leg : is 183.50799560546875 cm
Height predicted from lower_leg : is 163.63482666015625 cm
Height predicted from lower_leg : is 157.98666381835938 cm
Height predicted from lower_leg : is 161.75210571289062 cm
Height predicted from lower_leg : is 167.6094512939453 cm
Height predicted from lower_leg : is 163.84400939941406 cm
Height predicted from lower_leg : is 167.8186492919922 cm
Height predicted from lower_leg : is 166.14511108398438 cm
Height predicted from lower_leg : is 160.28778076171875 cm
Height predicted from lower_leg : is 168.86460876464844 cm
Height predicted from lower_leg : is 160.91534423828125 cm
Height predicted from lower_leg : is 164.68078

Height predicted from lower_leg : is 156.5223388671875 cm
Height predicted from lower_leg : is 161.9613037109375 cm
Height predicted from lower_leg : is 168.44622802734375 cm
Height predicted from lower_leg : is 162.5888671875 cm
Height predicted from lower_leg : is 170.1197509765625 cm
Height predicted from lower_leg : is 165.7267303466797 cm
Height predicted from lower_leg : is 163.21644592285156 cm
Height predicted from lower_leg : is 173.46681213378906 cm
Height predicted from lower_leg : is 177.2322540283203 cm
Height predicted from lower_leg : is 165.93592834472656 cm
Height predicted from lower_leg : is 174.51275634765625 cm
Height predicted from lower_leg : is 161.12452697753906 cm
Height predicted from lower_leg : is 168.23703002929688 cm
Height predicted from lower_leg : is 172.211669921875 cm
Height predicted from lower_leg : is 183.08961486816406 cm
Height predicted from lower_leg : is 168.65541076660156 cm
Height predicted from lower_leg : is 167.19107055664062 cm
Height p

Height predicted from lower_leg : is 173.67599487304688 cm
Height predicted from lower_leg : is 166.77268981933594 cm
Height predicted from lower_leg : is 163.42562866210938 cm
Height predicted from lower_leg : is 166.77268981933594 cm
Height predicted from lower_leg : is 162.79806518554688 cm
Height predicted from lower_leg : is 164.2624053955078 cm
Height predicted from lower_leg : is 166.56349182128906 cm
Height predicted from lower_leg : is 167.19107055664062 cm
Height predicted from lower_leg : is 164.6807861328125 cm
Height predicted from lower_leg : is 167.4002685546875 cm
Height predicted from lower_leg : is 168.86460876464844 cm
Height predicted from lower_leg : is 166.14511108398438 cm
Height predicted from lower_leg : is 165.7267303466797 cm
Height predicted from lower_leg : is 170.1197509765625 cm
Height predicted from lower_leg : is 160.4969482421875 cm
Height predicted from lower_leg : is 172.211669921875 cm
Height predicted from lower_leg : is 174.51275634765625 cm
Heigh

Height predicted from lower_leg : is 165.7267303466797 cm
Height predicted from lower_leg : is 171.58409118652344 cm
Height predicted from lower_leg : is 168.44622802734375 cm
Height predicted from lower_leg : is 171.58409118652344 cm
Height predicted from lower_leg : is 166.35430908203125 cm
Height predicted from lower_leg : is 160.28778076171875 cm
Height predicted from lower_leg : is 177.02305603027344 cm
Height predicted from lower_leg : is 161.54290771484375 cm
Height predicted from lower_leg : is 163.63482666015625 cm
Height predicted from lower_leg : is 163.42562866210938 cm
Height predicted from lower_leg : is 164.6807861328125 cm
Height predicted from lower_leg : is 169.28297424316406 cm
Height predicted from lower_leg : is 167.4002685546875 cm
Height predicted from lower_leg : is 173.67599487304688 cm
Height predicted from lower_leg : is 180.5792999267578 cm
Height predicted from lower_leg : is 171.37489318847656 cm
Height predicted from lower_leg : is 161.75210571289062 cm
H

Height predicted from lower_leg : is 153.80284118652344 cm
Height predicted from lower_leg : is 168.02784729003906 cm
Height predicted from lower_leg : is 159.45098876953125 cm
Height predicted from lower_leg : is 176.39547729492188 cm
Height predicted from lower_leg : is 169.4921875 cm
Height predicted from lower_leg : is 163.0072479248047 cm
Height predicted from lower_leg : is 170.1197509765625 cm
Height predicted from lower_leg : is 172.8392333984375 cm
Height predicted from lower_leg : is 172.6300506591797 cm
Height predicted from lower_leg : is 172.211669921875 cm
Height predicted from lower_leg : is 164.2624053955078 cm
Height predicted from lower_leg : is 170.95651245117188 cm
Height predicted from lower_leg : is 167.4002685546875 cm
Height predicted from lower_leg : is 166.9818878173828 cm
Height predicted from lower_leg : is 165.09915161132812 cm
Height predicted from lower_leg : is 161.33372497558594 cm
Height predicted from lower_leg : is 159.45098876953125 cm
Height predic

...

Height predicted from lower_leg : is 171.58409118652344 cm
Height predicted from lower_leg : is 160.4969482421875 cm
Height predicted from lower_leg : is 167.4002685546875 cm
Height predicted from lower_leg : is 166.14511108398438 cm
Height predicted from lower_leg : is 162.1704864501953 cm
Height predicted from lower_leg : is 166.56349182128906 cm
Height predicted from lower_leg : is 161.12452697753906 cm
Height predicted from lower_leg : is 163.21644592285156 cm
Height predicted from lower_leg : is 173.88519287109375 cm
Height predicted from lower_leg : is 171.16571044921875 cm
Height predicted from lower_leg : is 166.77268981933594 cm
Height predicted from lower_leg : is 168.44622802734375 cm
Height predicted from lower_leg : is 163.63482666015625 cm
Height predicted from lower_leg : is 166.9818878173828 cm
Height predicted from lower_leg : is 169.07379150390625 cm
Height predicted from lower_leg : is 168.86460876464844 cm
Height predicted from lower_leg : is 165.51754760742188 cm
H

Height predicted from lower_leg : is 164.47158813476562 cm
Height predicted from lower_leg : is 172.42086791992188 cm
Height predicted from lower_leg : is 171.16571044921875 cm
Height predicted from lower_leg : is 163.84400939941406 cm
Height predicted from lower_leg : is 166.14511108398438 cm
Height predicted from lower_leg : is 179.11495971679688 cm
Height predicted from lower_leg : is 166.35430908203125 cm
Height predicted from lower_leg : is 168.86460876464844 cm
Height predicted from lower_leg : is 174.93113708496094 cm
Height predicted from lower_leg : is 164.8899688720703 cm
Height predicted from lower_leg : is 172.42086791992188 cm
Height predicted from lower_leg : is 166.9818878173828 cm
Height predicted from lower_leg : is 171.16571044921875 cm
Height predicted from lower_leg : is 163.63482666015625 cm
Height predicted from lower_leg : is 159.45098876953125 cm
Height predicted from lower_leg : is 158.82342529296875 cm
Height predicted from lower_leg : is 165.7267303466797 cm


Height predicted from lower_leg : is 178.06903076171875 cm
Height predicted from lower_leg : is 175.34954833984375 cm
Height predicted from lower_leg : is 174.72195434570312 cm
Height predicted from lower_leg : is 169.4921875 cm
Height predicted from lower_leg : is 174.51275634765625 cm
Height predicted from lower_leg : is 158.61424255371094 cm
Height predicted from lower_leg : is 161.75210571289062 cm
Height predicted from lower_leg : is 169.4921875 cm
Height predicted from lower_leg : is 159.6602020263672 cm
Height predicted from lower_leg : is 169.7013702392578 cm
Height predicted from lower_leg : is 163.42562866210938 cm
Height predicted from lower_leg : is 165.93592834472656 cm
Height predicted from lower_leg : is 164.6807861328125 cm
Height predicted from lower_leg : is 164.47158813476562 cm
Height predicted from lower_leg : is 159.6602020263672 cm
Height predicted from lower_leg : is 168.23703002929688 cm
Height predicted from lower_leg : is 168.23703002929688 cm
Height predicte

Height predicted from lower_leg : is 170.1197509765625 cm
Height predicted from lower_leg : is 163.42562866210938 cm
Height predicted from lower_leg : is 167.4002685546875 cm
Height predicted from lower_leg : is 170.53814697265625 cm
Height predicted from lower_leg : is 173.04843139648438 cm
Height predicted from lower_leg : is 170.1197509765625 cm
Height predicted from lower_leg : is 165.7267303466797 cm
Height predicted from lower_leg : is 164.8899688720703 cm
Height predicted from lower_leg : is 166.77268981933594 cm
Height predicted from lower_leg : is 169.07379150390625 cm
Height predicted from lower_leg : is 176.81387329101562 cm
Height predicted from lower_leg : is 166.56349182128906 cm
Height predicted from lower_leg : is 166.56349182128906 cm
Height predicted from lower_leg : is 165.30836486816406 cm
Height predicted from lower_leg : is 161.33372497558594 cm
Height predicted from lower_leg : is 160.91534423828125 cm
Height predicted from lower_leg : is 166.9818878173828 cm
Hei

Height predicted from lower_leg : is 183.50799560546875 cm
Height predicted from lower_leg : is 186.22747802734375 cm
Height predicted from lower_leg : is 183.50799560546875 cm
Height predicted from lower_leg : is 179.95175170898438 cm
Height predicted from lower_leg : is 170.3289337158203 cm
Height predicted from lower_leg : is 177.85983276367188 cm
Height predicted from lower_leg : is 182.8804168701172 cm
Height predicted from lower_leg : is 178.6965789794922 cm
Height predicted from lower_leg : is 174.72195434570312 cm
Height predicted from lower_leg : is 179.11495971679688 cm
Height predicted from lower_leg : is 175.14035034179688 cm
Height predicted from lower_leg : is 165.51754760742188 cm
Height predicted from lower_leg : is 177.44143676757812 cm
Height predicted from lower_leg : is 175.9770965576172 cm
Height predicted from lower_leg : is 174.09439086914062 cm
Height predicted from lower_leg : is 177.2322540283203 cm
Height predicted from lower_leg : is 178.90577697753906 cm
He

# Evaluation Upper Leg (Femur) 

In [44]:
dataset_path ='data/nhannes.csv'
column_names = ['INDEX', 'HEIGHT', 'UPPER_LEG']


raw_dataset = pd.read_csv(dataset_path, names=column_names)

dataset = raw_dataset.copy()

x = dataset['UPPER_LEG']
y = dataset['HEIGHT']

x_train = []

for index, value in enumerate(x):
    if (value != 'upper_leg') and (float(value) > 10.0):
        x_train.append(float(value))

    
x_train_float = np.array(x_train[1:len(x_train)-3000])
print('Samples: ', len(x_train_float))


mae = 0
mse = 0
for index, femur in enumerate(x_train_float):
    
    upper_leg = femur
    flag = 'upper_leg'  # 174.8

    x1 = torch.from_numpy(np.array([upper_leg], dtype='float32'))
    x1.view((1,1))
    x, pred_LL, pred_UL, pred_LA, pred_UA = model(x1/55.5, flag)  
    mae += abs(pred_UL - femur)
    mse += (pred_UL - femur)**2
    
mae/=index+1
mse/=index+1

print('Mean Absolute Error on ', index, 'samples :', mae.item())
print('Mean Squared Error on ', index, 'samples :', np.sqrt(mse.item()))


Samples:  26045
Height predicted from upper_leg : is 147.40814208984375 cm
Height predicted from upper_leg : is 149.86605834960938 cm
Height predicted from upper_leg : is 175.98277282714844 cm
Height predicted from upper_leg : is 175.77101135253906 cm
Height predicted from upper_leg : is 158.60537719726562 cm
Height predicted from upper_leg : is 152.59710693359375 cm
Height predicted from upper_leg : is 154.50881958007812 cm
Height predicted from upper_leg : is 168.16397094726562 cm
Height predicted from upper_leg : is 159.9709014892578 cm
Height predicted from upper_leg : is 169.418701171875 cm
Height predicted from upper_leg : is 150.95851135253906 cm
Height predicted from upper_leg : is 151.7777862548828 cm
Height predicted from upper_leg : is 160.2439727783203 cm
Height predicted from upper_leg : is 165.43295288085938 cm
Height predicted from upper_leg : is 170.4774169921875 cm
Height predicted from upper_leg : is 150.4122772216797 cm
Height predicted from upper_leg : is 166.525360

Height predicted from upper_leg : is 165.97915649414062 cm
Height predicted from upper_leg : is 173.65357971191406 cm
Height predicted from upper_leg : is 172.59486389160156 cm
Height predicted from upper_leg : is 184.4525146484375 cm
Height predicted from upper_leg : is 164.3405303955078 cm
Height predicted from upper_leg : is 161.06329345703125 cm
Height predicted from upper_leg : is 167.61776733398438 cm
Height predicted from upper_leg : is 162.1557159423828 cm
Height predicted from upper_leg : is 172.59486389160156 cm
Height predicted from upper_leg : is 161.33641052246094 cm
Height predicted from upper_leg : is 146.31573486328125 cm
Height predicted from upper_leg : is 174.71229553222656 cm
Height predicted from upper_leg : is 161.33641052246094 cm
Height predicted from upper_leg : is 153.96261596679688 cm
Height predicted from upper_leg : is 154.78192138671875 cm
Height predicted from upper_leg : is 186.78172302246094 cm
Height predicted from upper_leg : is 162.70191955566406 cm


Height predicted from upper_leg : is 164.0674285888672 cm
Height predicted from upper_leg : is 169.84217834472656 cm
Height predicted from upper_leg : is 159.4246826171875 cm
Height predicted from upper_leg : is 150.13919067382812 cm
Height predicted from upper_leg : is 162.70191955566406 cm
Height predicted from upper_leg : is 153.41641235351562 cm
Height predicted from upper_leg : is 175.77101135253906 cm
Height predicted from upper_leg : is 165.43295288085938 cm
Height predicted from upper_leg : is 159.9709014892578 cm
Height predicted from upper_leg : is 170.90090942382812 cm
Height predicted from upper_leg : is 162.1557159423828 cm
Height predicted from upper_leg : is 155.05502319335938 cm
Height predicted from upper_leg : is 174.28880310058594 cm
Height predicted from upper_leg : is 154.50881958007812 cm
Height predicted from upper_leg : is 158.332275390625 cm
Height predicted from upper_leg : is 159.4246826171875 cm
Height predicted from upper_leg : is 165.7060546875 cm
Height p

Height predicted from upper_leg : is 153.96261596679688 cm
Height predicted from upper_leg : is 159.9709014892578 cm
Height predicted from upper_leg : is 136.55328369140625 cm
Height predicted from upper_leg : is 143.584716796875 cm
Height predicted from upper_leg : is 157.7860565185547 cm
Height predicted from upper_leg : is 161.6094970703125 cm
Height predicted from upper_leg : is 152.87020874023438 cm
Height predicted from upper_leg : is 144.95021057128906 cm
Height predicted from upper_leg : is 152.32400512695312 cm
Height predicted from upper_leg : is 160.51708984375 cm
Height predicted from upper_leg : is 155.60125732421875 cm
Height predicted from upper_leg : is 166.52536010742188 cm
Height predicted from upper_leg : is 140.85366821289062 cm
Height predicted from upper_leg : is 159.15158081054688 cm
Height predicted from upper_leg : is 153.14329528808594 cm
Height predicted from upper_leg : is 149.31985473632812 cm
Height predicted from upper_leg : is 169.63043212890625 cm
Heigh

Height predicted from upper_leg : is 159.9709014892578 cm
Height predicted from upper_leg : is 174.71229553222656 cm
Height predicted from upper_leg : is 171.5361328125 cm
Height predicted from upper_leg : is 174.71229553222656 cm
Height predicted from upper_leg : is 162.70191955566406 cm
Height predicted from upper_leg : is 158.60537719726562 cm
Height predicted from upper_leg : is 158.60537719726562 cm
Height predicted from upper_leg : is 174.07705688476562 cm
Height predicted from upper_leg : is 150.95851135253906 cm
Height predicted from upper_leg : is 145.76950073242188 cm
Height predicted from upper_leg : is 156.42054748535156 cm
Height predicted from upper_leg : is 155.05502319335938 cm
Height predicted from upper_leg : is 141.39988708496094 cm
Height predicted from upper_leg : is 172.80661010742188 cm
Height predicted from upper_leg : is 162.1557159423828 cm
Height predicted from upper_leg : is 159.4246826171875 cm
Height predicted from upper_leg : is 174.28880310058594 cm
Heig

...

Height predicted from upper_leg : is 161.8826141357422 cm
Height predicted from upper_leg : is 168.71018981933594 cm
Height predicted from upper_leg : is 176.82974243164062 cm
Height predicted from upper_leg : is 147.68124389648438 cm
Height predicted from upper_leg : is 166.7984619140625 cm
Height predicted from upper_leg : is 165.97915649414062 cm
Height predicted from upper_leg : is 155.87432861328125 cm
Height predicted from upper_leg : is 155.05502319335938 cm
Height predicted from upper_leg : is 155.32814025878906 cm
Height predicted from upper_leg : is 167.61776733398438 cm
Height predicted from upper_leg : is 159.9709014892578 cm
Height predicted from upper_leg : is 153.96261596679688 cm
Height predicted from upper_leg : is 169.20693969726562 cm
Height predicted from upper_leg : is 166.2522430419922 cm
Height predicted from upper_leg : is 161.33641052246094 cm
Height predicted from upper_leg : is 150.95851135253906 cm
Height predicted from upper_leg : is 147.68124389648438 cm
H

Height predicted from upper_leg : is 152.32400512695312 cm
Height predicted from upper_leg : is 171.95962524414062 cm
Height predicted from upper_leg : is 169.20693969726562 cm
Height predicted from upper_leg : is 170.4774169921875 cm
Height predicted from upper_leg : is 155.05502319335938 cm
Height predicted from upper_leg : is 166.7984619140625 cm
Height predicted from upper_leg : is 175.77101135253906 cm
Height predicted from upper_leg : is 170.4774169921875 cm
Height predicted from upper_leg : is 171.32440185546875 cm
Height predicted from upper_leg : is 153.14329528808594 cm
Height predicted from upper_leg : is 149.5929718017578 cm
Height predicted from upper_leg : is 164.6136474609375 cm
Height predicted from upper_leg : is 168.16397094726562 cm
Height predicted from upper_leg : is 161.33641052246094 cm
Height predicted from upper_leg : is 176.82974243164062 cm
Height predicted from upper_leg : is 140.85366821289062 cm
Height predicted from upper_leg : is 168.43707275390625 cm
He

Height predicted from upper_leg : is 140.0343475341797 cm
Height predicted from upper_leg : is 161.8826141357422 cm
Height predicted from upper_leg : is 173.4418487548828 cm
Height predicted from upper_leg : is 165.97915649414062 cm
Height predicted from upper_leg : is 161.8826141357422 cm
Height predicted from upper_leg : is 158.60537719726562 cm
Height predicted from upper_leg : is 159.9709014892578 cm
Height predicted from upper_leg : is 172.59486389160156 cm
Height predicted from upper_leg : is 158.60537719726562 cm
Height predicted from upper_leg : is 156.14744567871094 cm
Height predicted from upper_leg : is 169.84217834472656 cm
Height predicted from upper_leg : is 170.4774169921875 cm
Height predicted from upper_leg : is 173.2301025390625 cm
Height predicted from upper_leg : is 172.59486389160156 cm
Height predicted from upper_leg : is 151.7777862548828 cm
Height predicted from upper_leg : is 165.15985107421875 cm
Height predicted from upper_leg : is 167.07156372070312 cm
Heigh

Height predicted from upper_leg : is 167.07156372070312 cm
Height predicted from upper_leg : is 180.85289001464844 cm
Height predicted from upper_leg : is 186.3582305908203 cm
Height predicted from upper_leg : is 164.0674285888672 cm
Height predicted from upper_leg : is 163.52122497558594 cm
Height predicted from upper_leg : is 158.05917358398438 cm
Height predicted from upper_leg : is 144.1309051513672 cm
Height predicted from upper_leg : is 166.52536010742188 cm
Height predicted from upper_leg : is 177.4649658203125 cm
Height predicted from upper_leg : is 164.0674285888672 cm
Height predicted from upper_leg : is 164.0674285888672 cm
Height predicted from upper_leg : is 156.14744567871094 cm
Height predicted from upper_leg : is 162.42881774902344 cm
Height predicted from upper_leg : is 168.98329162597656 cm
Height predicted from upper_leg : is 170.4774169921875 cm
Height predicted from upper_leg : is 147.68124389648438 cm
Height predicted from upper_leg : is 154.50881958007812 cm
Heig

Height predicted from upper_leg : is 154.50881958007812 cm
Height predicted from upper_leg : is 174.71229553222656 cm
Height predicted from upper_leg : is 159.9709014892578 cm
Height predicted from upper_leg : is 168.71018981933594 cm
Height predicted from upper_leg : is 151.7777862548828 cm
Height predicted from upper_leg : is 148.5005340576172 cm
Height predicted from upper_leg : is 167.61776733398438 cm
Height predicted from upper_leg : is 171.5361328125 cm
Height predicted from upper_leg : is 170.05392456054688 cm
Height predicted from upper_leg : is 159.9709014892578 cm
Height predicted from upper_leg : is 155.87432861328125 cm
Height predicted from upper_leg : is 152.05088806152344 cm
Height predicted from upper_leg : is 158.60537719726562 cm
Height predicted from upper_leg : is 149.31985473632812 cm
Height predicted from upper_leg : is 162.70191955566406 cm
Height predicted from upper_leg : is 161.06329345703125 cm
Mean Absolute Error on  26044 samples : 9.15005874633789
Mean Sq

# Evaluation Lower Arm (Ulna)

In [48]:
mu , sigma = 250, 7

x_train_ulna = np.random.normal(mu, sigma, 15000)

noise = np.random.normal(0, 5, x_train_ulna.shape)
x_train_ulna = x_train_ulna + noise

print(len(x_train_ulna))


mae = 0
mse = 0
for index, ulna in enumerate(x_train_ulna):
    
    lower_arm = ulna
    flag = 'lower_arm'  

    x1 = torch.from_numpy(np.array([lower_arm], dtype='float32'))

    x1.view((1,1))    
    x, pred_LL, pred_UL, pred_LA, pred_UA = model(x1/280.15, flag)
    mae += abs(pred_LA - ulna/10)
    mse += (pred_LA - ulna/10)**2
    
mae/=index+1
mse/=index+1

print('Mean Absolute Error on ', index, 'samples :', mae.item())
print('Mean Squared Error on ', index, 'samples :', np.sqrt(mse.item()))


15000
Height predicted from lower_arm : is 183.2281951904297 cm
Height predicted from lower_arm : is 184.28048706054688 cm
Height predicted from lower_arm : is 191.55650329589844 cm
Height predicted from lower_arm : is 183.57167053222656 cm
Height predicted from lower_arm : is 168.99534606933594 cm
Height predicted from lower_arm : is 175.82102966308594 cm
Height predicted from lower_arm : is 181.91807556152344 cm
Height predicted from lower_arm : is 172.5420684814453 cm
Height predicted from lower_arm : is 176.12344360351562 cm
Height predicted from lower_arm : is 171.33900451660156 cm
Height predicted from lower_arm : is 184.54562377929688 cm
Height predicted from lower_arm : is 169.03195190429688 cm
Height predicted from lower_arm : is 188.26116943359375 cm
Height predicted from lower_arm : is 181.7406005859375 cm
Height predicted from lower_arm : is 176.98843383789062 cm
Height predicted from lower_arm : is 183.84034729003906 cm
Height predicted from lower_arm : is 176.880020141601

Height predicted from lower_arm : is 179.43165588378906 cm
Height predicted from lower_arm : is 173.3882598876953 cm
Height predicted from lower_arm : is 182.41981506347656 cm
Height predicted from lower_arm : is 180.18865966796875 cm
Height predicted from lower_arm : is 174.97451782226562 cm
Height predicted from lower_arm : is 177.9723358154297 cm
Height predicted from lower_arm : is 175.4661102294922 cm
Height predicted from lower_arm : is 174.6335906982422 cm
Height predicted from lower_arm : is 183.4935760498047 cm
Height predicted from lower_arm : is 180.65855407714844 cm
Height predicted from lower_arm : is 174.65525817871094 cm
Height predicted from lower_arm : is 180.93116760253906 cm
Height predicted from lower_arm : is 180.1795196533203 cm
Height predicted from lower_arm : is 172.96102905273438 cm
Height predicted from lower_arm : is 182.22366333007812 cm
Height predicted from lower_arm : is 164.24655151367188 cm
Height predicted from lower_arm : is 184.8011016845703 cm
Heig

Height predicted from lower_arm : is 173.46044921875 cm
Height predicted from lower_arm : is 178.4731903076172 cm
Height predicted from lower_arm : is 174.3047637939453 cm
Height predicted from lower_arm : is 172.05557250976562 cm
Height predicted from lower_arm : is 180.48435974121094 cm
Height predicted from lower_arm : is 176.134521484375 cm
Height predicted from lower_arm : is 176.49154663085938 cm
Height predicted from lower_arm : is 182.04147338867188 cm
Height predicted from lower_arm : is 172.3303985595703 cm
Height predicted from lower_arm : is 184.74603271484375 cm
Height predicted from lower_arm : is 181.6417236328125 cm
Height predicted from lower_arm : is 175.41676330566406 cm
Height predicted from lower_arm : is 180.97698974609375 cm
Height predicted from lower_arm : is 175.92349243164062 cm
Height predicted from lower_arm : is 176.71316528320312 cm
Height predicted from lower_arm : is 181.98391723632812 cm
Height predicted from lower_arm : is 190.72007751464844 cm
Height

Height predicted from lower_arm : is 180.5802001953125 cm
Height predicted from lower_arm : is 184.47154235839844 cm
Height predicted from lower_arm : is 179.72227478027344 cm
Height predicted from lower_arm : is 173.1796417236328 cm
Height predicted from lower_arm : is 164.69683837890625 cm
Height predicted from lower_arm : is 191.55650329589844 cm
Height predicted from lower_arm : is 178.62139892578125 cm
Height predicted from lower_arm : is 169.18435668945312 cm
Height predicted from lower_arm : is 172.661376953125 cm
Height predicted from lower_arm : is 181.22003173828125 cm
Height predicted from lower_arm : is 179.24131774902344 cm
Height predicted from lower_arm : is 179.72158813476562 cm
Height predicted from lower_arm : is 187.6685791015625 cm
Height predicted from lower_arm : is 189.28509521484375 cm
Height predicted from lower_arm : is 176.0764923095703 cm
Height predicted from lower_arm : is 180.1435546875 cm
Height predicted from lower_arm : is 176.2516326904297 cm
Height p

Height predicted from lower_arm : is 184.6992950439453 cm
Height predicted from lower_arm : is 173.32431030273438 cm
Height predicted from lower_arm : is 174.46705627441406 cm
Height predicted from lower_arm : is 180.6776885986328 cm
Height predicted from lower_arm : is 178.84725952148438 cm
Height predicted from lower_arm : is 180.38388061523438 cm
Height predicted from lower_arm : is 171.0334930419922 cm
Height predicted from lower_arm : is 179.78733825683594 cm
Height predicted from lower_arm : is 191.55650329589844 cm
Height predicted from lower_arm : is 176.95303344726562 cm
Height predicted from lower_arm : is 170.03187561035156 cm
Height predicted from lower_arm : is 181.34849548339844 cm
Height predicted from lower_arm : is 180.2806396484375 cm
Height predicted from lower_arm : is 180.74171447753906 cm
Height predicted from lower_arm : is 177.5968780517578 cm
Height predicted from lower_arm : is 175.2023468017578 cm
Height predicted from lower_arm : is 177.13954162597656 cm
Hei

...

Height predicted from lower_arm : is 169.4987335205078 cm
Height predicted from lower_arm : is 177.1101837158203 cm
Height predicted from lower_arm : is 183.9634246826172 cm
Height predicted from lower_arm : is 172.2180633544922 cm
Height predicted from lower_arm : is 181.54916381835938 cm
Height predicted from lower_arm : is 181.8988494873047 cm
Height predicted from lower_arm : is 176.55184936523438 cm
Height predicted from lower_arm : is 169.3870849609375 cm
Height predicted from lower_arm : is 183.02122497558594 cm
Height predicted from lower_arm : is 179.74008178710938 cm
Height predicted from lower_arm : is 181.52764892578125 cm
Height predicted from lower_arm : is 175.30096435546875 cm
Height predicted from lower_arm : is 178.28994750976562 cm
Height predicted from lower_arm : is 180.3248291015625 cm
Height predicted from lower_arm : is 181.31085205078125 cm
Height predicted from lower_arm : is 186.7944793701172 cm
Height predicted from lower_arm : is 174.37921142578125 cm
Heigh

Height predicted from lower_arm : is 183.6952362060547 cm
Height predicted from lower_arm : is 171.0519256591797 cm
Height predicted from lower_arm : is 181.0797119140625 cm
Height predicted from lower_arm : is 166.86306762695312 cm
Height predicted from lower_arm : is 178.3386688232422 cm
Height predicted from lower_arm : is 178.9938507080078 cm
Height predicted from lower_arm : is 179.4712371826172 cm
Height predicted from lower_arm : is 179.1348876953125 cm
Height predicted from lower_arm : is 187.9217529296875 cm
Height predicted from lower_arm : is 184.46884155273438 cm
Height predicted from lower_arm : is 175.17239379882812 cm
Height predicted from lower_arm : is 171.40142822265625 cm
Height predicted from lower_arm : is 182.81088256835938 cm
Height predicted from lower_arm : is 186.63278198242188 cm
Height predicted from lower_arm : is 169.65185546875 cm
Height predicted from lower_arm : is 171.4205322265625 cm
Height predicted from lower_arm : is 181.95550537109375 cm
Height pr

Height predicted from lower_arm : is 180.8328857421875 cm
Height predicted from lower_arm : is 169.9524383544922 cm
Height predicted from lower_arm : is 175.0296173095703 cm
Height predicted from lower_arm : is 174.53366088867188 cm
Height predicted from lower_arm : is 179.2990264892578 cm
Height predicted from lower_arm : is 173.8917999267578 cm
Height predicted from lower_arm : is 184.548095703125 cm
Height predicted from lower_arm : is 180.31105041503906 cm
Height predicted from lower_arm : is 166.49301147460938 cm
Height predicted from lower_arm : is 175.2019805908203 cm
Height predicted from lower_arm : is 166.6679229736328 cm
Height predicted from lower_arm : is 182.27560424804688 cm
Height predicted from lower_arm : is 180.90377807617188 cm
Height predicted from lower_arm : is 176.914306640625 cm
Height predicted from lower_arm : is 178.6140899658203 cm
Height predicted from lower_arm : is 178.9597625732422 cm
Height predicted from lower_arm : is 177.53973388671875 cm
Height pre

Height predicted from lower_arm : is 182.1482696533203 cm
Height predicted from lower_arm : is 173.56448364257812 cm
Height predicted from lower_arm : is 172.98020935058594 cm
Height predicted from lower_arm : is 177.71597290039062 cm
Height predicted from lower_arm : is 178.36907958984375 cm
Height predicted from lower_arm : is 191.55650329589844 cm
Height predicted from lower_arm : is 176.4984893798828 cm
Height predicted from lower_arm : is 185.51429748535156 cm
Height predicted from lower_arm : is 187.21377563476562 cm
Height predicted from lower_arm : is 172.3843231201172 cm
Height predicted from lower_arm : is 165.37677001953125 cm
Height predicted from lower_arm : is 177.5856475830078 cm
Height predicted from lower_arm : is 183.645751953125 cm
Height predicted from lower_arm : is 176.87403869628906 cm
Height predicted from lower_arm : is 180.52516174316406 cm
Height predicted from lower_arm : is 188.71095275878906 cm
Height predicted from lower_arm : is 178.11305236816406 cm
Hei

Height predicted from lower_arm : is 184.229248046875 cm
Height predicted from lower_arm : is 178.28387451171875 cm
Height predicted from lower_arm : is 185.0107879638672 cm
Height predicted from lower_arm : is 180.35687255859375 cm
Height predicted from lower_arm : is 178.7560272216797 cm
Height predicted from lower_arm : is 187.0126190185547 cm
Height predicted from lower_arm : is 183.71572875976562 cm
Height predicted from lower_arm : is 177.9287109375 cm
Height predicted from lower_arm : is 174.03286743164062 cm
Height predicted from lower_arm : is 182.5065155029297 cm
Height predicted from lower_arm : is 180.2982940673828 cm
Height predicted from lower_arm : is 177.62005615234375 cm
Height predicted from lower_arm : is 174.87489318847656 cm
Height predicted from lower_arm : is 181.9474639892578 cm
Height predicted from lower_arm : is 175.03829956054688 cm
Height predicted from lower_arm : is 180.4297637939453 cm
Height predicted from lower_arm : is 188.833740234375 cm
Height predi

# Evaluation Upper Arm (Humerus)

In [49]:
mu , sigma = 294, 5

x_train_humerus = np.random.normal(mu, sigma, 15000)

noise = np.random.normal(0, 5, x_train_humerus.shape)
x_train_humerus = x_train_humerus + noise

print(len(x_train_humerus))


mae = 0
mse = 0
for index, humerus in enumerate(x_train_humerus):
    upper_arm = humerus
    flag = 'upper_arm'  # 182.67

    x1 = torch.from_numpy(np.array([upper_arm], dtype='float32'))
    x1.view((1,1))        
    x, pred_LL, pred_UL, pred_LA, pred_UA = model(x1/389.098, flag)
    mae += abs(pred_UA - humerus/10)
    mse += (pred_UA - humerus/10)**2
    
mae/=index+1
mse/=index+1


print('Mean Absolute Error on ', index, 'samples :', mae.item())
print('Mean Squared Error on ', index, 'samples :', np.sqrt(mse.item()))


15000
Height predicted from upper_arm : is 168.71717834472656 cm
Height predicted from upper_arm : is 170.65550231933594 cm
Height predicted from upper_arm : is 172.59974670410156 cm
Height predicted from upper_arm : is 167.77755737304688 cm
Height predicted from upper_arm : is 172.1949462890625 cm
Height predicted from upper_arm : is 171.58746337890625 cm
Height predicted from upper_arm : is 169.30458068847656 cm
Height predicted from upper_arm : is 168.68899536132812 cm
Height predicted from upper_arm : is 168.2849884033203 cm
Height predicted from upper_arm : is 168.77334594726562 cm
Height predicted from upper_arm : is 165.5866241455078 cm
Height predicted from upper_arm : is 171.88262939453125 cm
Height predicted from upper_arm : is 171.4746551513672 cm
Height predicted from upper_arm : is 168.6546173095703 cm
Height predicted from upper_arm : is 169.5361785888672 cm
Height predicted from upper_arm : is 168.41648864746094 cm
Height predicted from upper_arm : is 170.10935974121094 

Height predicted from upper_arm : is 168.5348358154297 cm
Height predicted from upper_arm : is 168.33621215820312 cm
Height predicted from upper_arm : is 171.2154083251953 cm
Height predicted from upper_arm : is 169.32421875 cm
Height predicted from upper_arm : is 170.8334197998047 cm
Height predicted from upper_arm : is 169.02122497558594 cm
Height predicted from upper_arm : is 168.35006713867188 cm
Height predicted from upper_arm : is 168.70285034179688 cm
Height predicted from upper_arm : is 168.67864990234375 cm
Height predicted from upper_arm : is 169.704345703125 cm
Height predicted from upper_arm : is 169.99520874023438 cm
Height predicted from upper_arm : is 169.48513793945312 cm
Height predicted from upper_arm : is 168.9215087890625 cm
Height predicted from upper_arm : is 168.13499450683594 cm
Height predicted from upper_arm : is 166.53070068359375 cm
Height predicted from upper_arm : is 170.8596649169922 cm
Height predicted from upper_arm : is 167.8507843017578 cm
Height pred

Height predicted from upper_arm : is 168.70620727539062 cm
Height predicted from upper_arm : is 168.1673126220703 cm
Height predicted from upper_arm : is 169.243408203125 cm
Height predicted from upper_arm : is 170.54539489746094 cm
Height predicted from upper_arm : is 170.03802490234375 cm
Height predicted from upper_arm : is 168.45018005371094 cm
Height predicted from upper_arm : is 168.80213928222656 cm
Height predicted from upper_arm : is 169.0760040283203 cm
Height predicted from upper_arm : is 170.31585693359375 cm
Height predicted from upper_arm : is 170.04017639160156 cm
Height predicted from upper_arm : is 170.5352020263672 cm
Height predicted from upper_arm : is 168.9078826904297 cm
Height predicted from upper_arm : is 167.55880737304688 cm
Height predicted from upper_arm : is 167.97097778320312 cm
Height predicted from upper_arm : is 169.1594696044922 cm
Height predicted from upper_arm : is 171.90203857421875 cm
Height predicted from upper_arm : is 171.1094970703125 cm
Heigh

Height predicted from upper_arm : is 169.3755340576172 cm
Height predicted from upper_arm : is 169.86029052734375 cm
Height predicted from upper_arm : is 170.09286499023438 cm
Height predicted from upper_arm : is 168.56175231933594 cm
Height predicted from upper_arm : is 169.52920532226562 cm
Height predicted from upper_arm : is 169.63818359375 cm
Height predicted from upper_arm : is 168.9298553466797 cm
Height predicted from upper_arm : is 168.49652099609375 cm
Height predicted from upper_arm : is 169.1259307861328 cm
Height predicted from upper_arm : is 170.0861358642578 cm
Height predicted from upper_arm : is 168.12274169921875 cm
Height predicted from upper_arm : is 172.41818237304688 cm
Height predicted from upper_arm : is 170.1547393798828 cm
Height predicted from upper_arm : is 170.44467163085938 cm
Height predicted from upper_arm : is 169.757080078125 cm
Height predicted from upper_arm : is 170.34689331054688 cm
Height predicted from upper_arm : is 167.61045837402344 cm
Height 

Height predicted from upper_arm : is 171.15017700195312 cm
Height predicted from upper_arm : is 168.8370361328125 cm
Height predicted from upper_arm : is 169.6477508544922 cm
Height predicted from upper_arm : is 169.7257080078125 cm
Height predicted from upper_arm : is 170.73426818847656 cm
Height predicted from upper_arm : is 171.69529724121094 cm
Height predicted from upper_arm : is 166.72618103027344 cm
Height predicted from upper_arm : is 165.68983459472656 cm
Height predicted from upper_arm : is 168.9571990966797 cm
Height predicted from upper_arm : is 167.93446350097656 cm
Height predicted from upper_arm : is 168.9451446533203 cm
Height predicted from upper_arm : is 168.4538116455078 cm
Height predicted from upper_arm : is 168.5872344970703 cm
Height predicted from upper_arm : is 168.4886474609375 cm
Height predicted from upper_arm : is 169.69448852539062 cm
Height predicted from upper_arm : is 170.02003479003906 cm
Height predicted from upper_arm : is 168.70553588867188 cm
Heigh

...

Height predicted from upper_arm : is 170.305419921875 cm
Height predicted from upper_arm : is 169.39990234375 cm
Height predicted from upper_arm : is 170.48464965820312 cm
Height predicted from upper_arm : is 169.19728088378906 cm
Height predicted from upper_arm : is 169.44024658203125 cm
Height predicted from upper_arm : is 169.3394012451172 cm
Height predicted from upper_arm : is 167.4024658203125 cm
Height predicted from upper_arm : is 169.76951599121094 cm
Height predicted from upper_arm : is 171.1250762939453 cm
Height predicted from upper_arm : is 169.1264190673828 cm
Height predicted from upper_arm : is 170.95245361328125 cm
Height predicted from upper_arm : is 167.96844482421875 cm
Height predicted from upper_arm : is 169.83326721191406 cm
Height predicted from upper_arm : is 171.24871826171875 cm
Height predicted from upper_arm : is 170.0359344482422 cm
Height predicted from upper_arm : is 168.21400451660156 cm
Height predicted from upper_arm : is 168.91995239257812 cm
Height 

Height predicted from upper_arm : is 168.49232482910156 cm
Height predicted from upper_arm : is 170.6658172607422 cm
Height predicted from upper_arm : is 168.72402954101562 cm
Height predicted from upper_arm : is 171.17079162597656 cm
Height predicted from upper_arm : is 167.21182250976562 cm
Height predicted from upper_arm : is 167.19740295410156 cm
Height predicted from upper_arm : is 168.2389373779297 cm
Height predicted from upper_arm : is 167.83486938476562 cm
Height predicted from upper_arm : is 170.00901794433594 cm
Height predicted from upper_arm : is 167.5072784423828 cm
Height predicted from upper_arm : is 169.7369842529297 cm
Height predicted from upper_arm : is 171.17958068847656 cm
Height predicted from upper_arm : is 172.35641479492188 cm
Height predicted from upper_arm : is 167.92686462402344 cm
Height predicted from upper_arm : is 168.8654022216797 cm
Height predicted from upper_arm : is 170.28866577148438 cm
Height predicted from upper_arm : is 171.07443237304688 cm
He

Height predicted from upper_arm : is 169.45602416992188 cm
Height predicted from upper_arm : is 167.96656799316406 cm
Height predicted from upper_arm : is 170.00680541992188 cm
Height predicted from upper_arm : is 168.8211669921875 cm
Height predicted from upper_arm : is 172.19740295410156 cm
Height predicted from upper_arm : is 169.33396911621094 cm
Height predicted from upper_arm : is 168.20297241210938 cm
Height predicted from upper_arm : is 166.5980224609375 cm
Height predicted from upper_arm : is 171.44064331054688 cm
Height predicted from upper_arm : is 169.50733947753906 cm
Height predicted from upper_arm : is 169.45729064941406 cm
Height predicted from upper_arm : is 166.1112823486328 cm
Height predicted from upper_arm : is 171.64019775390625 cm
Height predicted from upper_arm : is 166.53173828125 cm
Height predicted from upper_arm : is 170.90509033203125 cm
Height predicted from upper_arm : is 170.78448486328125 cm
Height predicted from upper_arm : is 168.6558837890625 cm
Heig

Height predicted from upper_arm : is 171.0308074951172 cm
Height predicted from upper_arm : is 169.89170837402344 cm
Height predicted from upper_arm : is 169.8851776123047 cm
Height predicted from upper_arm : is 171.24880981445312 cm
Height predicted from upper_arm : is 168.39602661132812 cm
Height predicted from upper_arm : is 168.19444274902344 cm
Height predicted from upper_arm : is 171.10690307617188 cm
Height predicted from upper_arm : is 168.8667755126953 cm
Height predicted from upper_arm : is 170.16006469726562 cm
Height predicted from upper_arm : is 167.98414611816406 cm
Height predicted from upper_arm : is 169.2720184326172 cm
Height predicted from upper_arm : is 167.7048797607422 cm
Height predicted from upper_arm : is 168.76116943359375 cm
Height predicted from upper_arm : is 167.7755889892578 cm
Height predicted from upper_arm : is 168.5871124267578 cm
Height predicted from upper_arm : is 169.92784118652344 cm
Height predicted from upper_arm : is 167.93057250976562 cm
Heig

Height predicted from upper_arm : is 169.7710418701172 cm
Height predicted from upper_arm : is 168.76820373535156 cm
Height predicted from upper_arm : is 169.7828826904297 cm
Height predicted from upper_arm : is 168.64341735839844 cm
Height predicted from upper_arm : is 168.70584106445312 cm
Height predicted from upper_arm : is 168.94284057617188 cm
Height predicted from upper_arm : is 168.40451049804688 cm
Height predicted from upper_arm : is 169.570068359375 cm
Height predicted from upper_arm : is 169.43263244628906 cm
Height predicted from upper_arm : is 167.74937438964844 cm
Height predicted from upper_arm : is 170.48304748535156 cm
Height predicted from upper_arm : is 170.53135681152344 cm
Height predicted from upper_arm : is 170.98545837402344 cm
Height predicted from upper_arm : is 169.32443237304688 cm
Height predicted from upper_arm : is 169.7056427001953 cm
Height predicted from upper_arm : is 170.28807067871094 cm
Height predicted from upper_arm : is 169.9822998046875 cm
Hei